In [1]:
import csv
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import time

import pandas as pd 
from IPython.display import display

# Build Review dataset using user and place indices

In [55]:
reviews = pd.read_csv("../datasets/google_local/reviews.csv")
display(reviews.info())
display(reviews.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11453845 entries, 0 to 11453844
Data columns (total 4 columns):
gPlusPlaceId      object
gPlusUserId       object
rating            float64
unixReviewTime    object
dtypes: float64(1), object(3)
memory usage: 349.5+ MB


None

,gPlusPlaceId,gPlusUserId,rating,unixReviewTime
0,108103314380004200232,100000010817154263736,3.0,1372686659
1,102194128241608748649,100000013500285534661,5.0,1342870724
2,101409858828175402384,100000021336848867366,5.0,1390653513
3,101477177500158511502,100000021336848867366,5.0,1389187706
4,106994170641063333085,100000021336848867366,4.0,1390486279


In [52]:
place_ids = list(reviews['gPlusPlaceId'].unique())
user_ids = list(reviews['gPlusUserId'].unique())

place_ids.sort()
user_ids.sort()

print(len(place_ids), place_ids[:3])
print(len(user_ids), user_ids[:3])

3116785 ['100000007848517518160', '100000027961183785551', '100000028989488322699']
5054567 ['100000010817154263736', '100000013500285534661', '100000021336848867366']


### Replace IDs with Indices

In [56]:
start = time.time()

from bisect import bisect_left 
  
def BinarySearch(a, x): 
    i = bisect_left(a, x) 
    if i != len(a) and a[i] == x: 
        return i 
    else: 
        return -1

for i in reviews.index:
    if i%100000==0:
        print(time.time()-start)
        start = time.time()
        print(i)
    pid = reviews.at[i,'gPlusPlaceId']
    uid = reviews.at[i,'gPlusUserId']

    reviews.at[i,'gPlusPlaceId'] = BinarySearch(place_ids,pid)
    reviews.at[i,'gPlusUserId'] = BinarySearch(user_ids,uid)


0.2108461856842041
0
5.007460594177246
100000
4.743478059768677
200000
4.939120531082153
300000
4.669917583465576
400000
5.119900226593018
500000
4.73084020614624
600000
4.80470085144043
700000
4.740260362625122
800000
4.993489503860474
900000
4.868492603302002
1000000
4.786747217178345
1100000
4.9740073680877686
1200000
4.781872272491455
1300000
5.226130247116089
1400000
4.79388427734375
1500000
4.857175827026367
1600000
4.732042551040649
1700000
4.788888931274414
1800000
4.728853464126587
1900000
4.73508620262146
2000000
4.781548023223877
2100000
4.7074689865112305
2200000
5.060934782028198
2300000
4.9958274364471436
2400000
5.124858140945435
2500000
4.90395712852478
2600000
5.271379709243774
2700000
5.298417568206787
2800000
5.109431505203247
2900000
4.716844081878662
3000000
4.904526472091675
3100000
4.850224494934082
3200000
5.091361045837402
3300000
4.997869491577148
3400000
4.9076759815216064
3500000
4.718376636505127
3600000
4.892521858215332
3700000
5.285653591156006
3800000
5

In [57]:
reviews[:10]

,gPlusPlaceId,gPlusUserId,rating,unixReviewTime
0,1368311,0,3.0,1372686659
1,370282,1,5.0,1342870724
2,237940,2,5.0,1390653513
3,249417,2,5.0,1389187706
4,1181533,2,4.0,1390486279
5,1257074,2,5.0,1390192729
6,1350634,2,5.0,1390825528
7,1919338,2,4.0,1390445561
8,2018069,2,4.0,1390387529
9,2348090,2,5.0,1392295463


### Save to files

In [60]:
fname = "../datasets/google_local/reviews_reformatted.csv"

print(reviews[:10])

reviews.to_csv(fname, encoding='utf-8', index=False)

  gPlusPlaceId gPlusUserId  rating unixReviewTime
0      1368311           0     3.0     1372686659
1       370282           1     5.0     1342870724
2       237940           2     5.0     1390653513
3       249417           2     5.0     1389187706
4      1181533           2     4.0     1390486279
5      1257074           2     5.0     1390192729
6      1350634           2     5.0     1390825528
7      1919338           2     4.0     1390445561
8      2018069           2     4.0     1390387529
9      2348090           2     5.0     1392295463


In [58]:
fname = "../datasets/google_local/user_reformatted.csv"

users = pd.Series(user_ids)

print(users[:10])

users.to_csv(fname, encoding='utf-8', index=True)

0    100000010817154263736
1    100000013500285534661
2    100000021336848867366
3    100000030557048145331
4    100000032416892623125
5    100000035085750632094
6    100000036174088924566
7    100000037905537879319
8    100000039815955317902
9    100000042779388982190
dtype: object


In [59]:
fname = "../datasets/google_local/place_reformatted.csv"

places = pd.Series(place_ids)

print(places[:10])

places.to_csv(fname, encoding='utf-8', index=True)

0    100000007848517518160
1    100000027961183785551
2    100000028989488322699
3    100000030371226927960
4    100000031133105350522
5    100000038035038453021
6    100000061163388376076
7    100000061963763467148
8    100000075541128264108
9    100000077955167136431
dtype: object


## Extract ('PlaceId','UserId','rating,'Time','num_reviews') --> reviews_freq.csv

In [2]:
reviews = pd.read_csv("../datasets/google_local/reviews_reformatted.csv")
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11453845 entries, 0 to 11453844
Data columns (total 4 columns):
gPlusPlaceId      int64
gPlusUserId       int64
rating            float64
unixReviewTime    object
dtypes: float64(1), int64(2), object(1)
memory usage: 349.5+ MB


In [4]:
freq = reviews['gPlusPlaceId'].value_counts()
sorted_freq = freq.sort_index()

sorted_freq[-15:]

3116770     1
3116771    36
3116772     8
3116773     1
3116774     2
3116775     3
3116776     1
3116777     1
3116778     2
3116779     5
3116780     1
3116781     1
3116782     4
3116783     4
3116784     8
Name: gPlusPlaceId, dtype: int64

In [9]:
reviews[reviews['gPlusPlaceId']==1368311]

,gPlusPlaceId,gPlusUserId,rating,unixReviewTime,num_reviews
0,1368311,0,3.0,1372686659,0
1045535,1368311,453510,5.0,1354888378,0
4736034,1368311,2075473,2.0,1352014612,0


In [6]:
reviews['num_reviews'] = 0

reviews.head()

,gPlusPlaceId,gPlusUserId,rating,unixReviewTime,num_reviews
0,1368311,0,3.0,1372686659,0
1,370282,1,5.0,1342870724,0
2,237940,2,5.0,1390653513,0
3,249417,2,5.0,1389187706,0
4,1181533,2,4.0,1390486279,0


In [10]:
from bisect import bisect_left 
  
def BinarySearch(a, x): 
    i = bisect_left(a, x) 
    if i != len(a) and a[i] == x: 
        return i 
    else: 
        return -1

freqs = list(sorted_freq)
    
start = time.time()

for i in reviews.index:
    if i%100000==0:
        print(time.time()-start)
        start = time.time()
        print(i)
    pid = reviews.at[i,'gPlusPlaceId']
    reviews.at[i,'num_reviews'] = freqs[pid]


0.1749417781829834
0
2.303776502609253
100000
2.3238842487335205
200000
2.2362613677978516
300000
2.2587454319000244
400000
2.2617833614349365
500000
2.274261713027954
600000
2.2817633152008057
700000
2.2457847595214844
800000
2.2641303539276123
900000
2.2977306842803955
1000000
2.299499988555908
1100000
2.2834768295288086
1200000
2.2872798442840576
1300000
2.2886526584625244
1400000
2.2413692474365234
1500000
2.4398388862609863
1600000
2.307511329650879
1700000
2.286322593688965
1800000
2.282982587814331
1900000
2.2772300243377686
2000000
2.2877540588378906
2100000
2.281782627105713
2200000
2.270968437194824
2300000
2.313129186630249
2400000
2.478689193725586
2500000
2.4002010822296143
2600000
2.3349761962890625
2700000
2.3845486640930176
2800000
2.2766404151916504
2900000
2.291806697845459
3000000
2.330415964126587
3100000
2.277099132537842
3200000
2.2733657360076904
3300000
2.3407680988311768
3400000
2.4201526641845703
3500000
2.3667373657226562
3600000
2.291761875152588
3700000
2.3

In [11]:
reviews.head()

,gPlusPlaceId,gPlusUserId,rating,unixReviewTime,num_reviews
0,1368311,0,3.0,1372686659,3
1,370282,1,5.0,1342870724,3
2,237940,2,5.0,1390653513,1
3,249417,2,5.0,1389187706,2
4,1181533,2,4.0,1390486279,1


In [15]:
# Basic QC

for i in range(5):
    index = random.randint(0, len(sorted_freq))
    freq = reviews.at[index,'num_reviews']
    idx = reviews.at[index,'gPlusPlaceId']
    print(idx, freq)
    display(reviews[reviews['gPlusPlaceId']==idx])

451999 3


,gPlusPlaceId,gPlusUserId,rating,unixReviewTime,num_reviews
1596933,451999,694891,3.0,1368389636,3
3918672,451999,1714305,5.0,1346976026,3
3919852,451999,1714919,4.0,1349390303,3


2493246 4


,gPlusPlaceId,gPlusUserId,rating,unixReviewTime,num_reviews
1668329,2493246,726216,1.0,1383849264,4
7544915,2493246,3312182,2.0,1368215140,4
9066961,2493246,3982170,2.0,1364830566,4
10900098,2493246,4803369,5.0,1363823129,4


238122 2


,gPlusPlaceId,gPlusUserId,rating,unixReviewTime,num_reviews
360277,238122,158487,4.0,1352581923,2
8453045,238122,3713272,5.0,1337243609,2


1988528 27


,gPlusPlaceId,gPlusUserId,rating,unixReviewTime,num_reviews
1548186,1988528,673813,3.0,1343317427,27
1734221,1988528,756162,5.0,1331741491,27
3024562,1988528,1320436,5.0,1355615302,27
3068098,1988528,1339667,2.0,1352630106,27
3954855,1988528,1730611,4.0,None,27
4265798,1988528,1866726,2.0,1352631142,27
4269762,1988528,1868512,4.0,1339253092,27
4559932,1988528,1996843,5.0,1294068319,27
4916655,1988528,2153645,4.0,1340819622,27
5190379,1988528,2275168,5.0,1374432793,27


2439702 5


,gPlusPlaceId,gPlusUserId,rating,unixReviewTime,num_reviews
558073,2439702,242559,4.0,1375057446,5
694533,2439702,300795,4.0,1372463436,5
3274283,2439702,1428589,4.0,1373414544,5
7071561,2439702,3102485,3.0,1362274879,5
8716651,2439702,3828882,5.0,1370977600,5


In [16]:
fname = "../datasets/google_local/reviews_freq.csv"

reviews.to_csv(fname, encoding='utf-8', index=False)

## ('placeId','userId','rating,'Unix_time','num_reviews','year','month','date' ) --> reviews_time_sorted.csv

This file will be used for temporal prediction.

In [17]:
reviews = pd.read_csv("../datasets/google_local/reviews_freq.csv")
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11453845 entries, 0 to 11453844
Data columns (total 5 columns):
gPlusPlaceId      int64
gPlusUserId       int64
rating            float64
unixReviewTime    object
num_reviews       int64
dtypes: float64(1), int64(3), object(1)
memory usage: 436.9+ MB


In [25]:
# Remove rows where unixReviewTime='None'
indexNames = reviews[reviews["unixReviewTime"]=='None'].index
reviews.drop(indexNames , inplace=True) 

reviews_timed = reviews.astype('int32')  # cast all data as int32
reviews_timesorted = reviews_timed.sort_values(by=["unixReviewTime"])  # sort in ascending time
reviews_timesorted = reviews_timesorted.reset_index(drop=True)  # Reset index 

display(reviews_timesorted.info())
display(reviews_timesorted.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10601852 entries, 0 to 10601851
Data columns (total 5 columns):
gPlusPlaceId      int32
gPlusUserId       int32
rating            int32
unixReviewTime    int32
num_reviews       int32
dtypes: int32(5)
memory usage: 202.2 MB


None

,gPlusPlaceId,gPlusUserId,rating,unixReviewTime,num_reviews
0,804813,2021440,4,662601600,9
1,1918972,389663,4,662601600,20
2,449452,2709545,4,662601600,15
3,942354,4936600,4,662601600,293
4,3063673,828378,5,662601600,11


In [26]:
# Add 3 columns
reviews_timesorted['year'] = 0
reviews_timesorted['month'] = 0
reviews_timesorted['day'] = 0

display(reviews_timesorted.head())

,gPlusPlaceId,gPlusUserId,rating,unixReviewTime,num_reviews,year,month,day
0,804813,2021440,4,662601600,9,0,0,0
1,1918972,389663,4,662601600,20,0,0,0
2,449452,2709545,4,662601600,15,0,0,0
3,942354,4936600,4,662601600,293,0,0,0
4,3063673,828378,5,662601600,11,0,0,0


In [27]:
start = time.time()

for i in reviews_timesorted.index:
    if i%100000==0:
        print(time.time()-start)
        start = time.time()
        print(i)
        
    TimeInt = reviews_timesorted.at[i,'unixReviewTime']
    timeStruct = time.gmtime(TimeInt)
    year, month, day = map(int,time.strftime("%Y %m %d",timeStruct).split())
    
    reviews_timesorted.at[i,'year'] = year
    reviews_timesorted.at[i,'month'] = month
    reviews_timesorted.at[i,'day'] = day

0.17203736305236816
0
4.825153827667236
100000
4.97382926940918
200000
4.850502967834473
300000
4.8605852127075195
400000
4.860478639602661
500000
5.001388072967529
600000
5.154032945632935
700000
4.922649621963501
800000
4.915227651596069
900000
4.917226552963257
1000000
4.951488971710205
1100000
4.928593158721924
1200000
5.0587756633758545
1300000
4.994058847427368
1400000
5.014569997787476
1500000
4.902041673660278
1600000
4.885256290435791
1700000
4.924117565155029
1800000
4.944265604019165
1900000
4.981187343597412
2000000
4.868201494216919
2100000
4.893624782562256
2200000
4.963334798812866
2300000
4.944225788116455
2400000
4.989045143127441
2500000
4.946710109710693
2600000
5.009919166564941
2700000
4.897155284881592
2800000
4.870684623718262
2900000
4.9110894203186035
3000000
4.909749507904053
3100000
4.95796537399292
3200000
4.971311807632446
3300000
4.928941965103149
3400000
4.940640449523926
3500000
4.9378275871276855
3600000
4.962620258331299
3700000
4.989256381988525
38000

In [29]:
display(reviews_timesorted.tail())

,gPlusPlaceId,gPlusUserId,rating,unixReviewTime,num_reviews,year,month,day
10601847,1938164,3368185,5,1396078130,4,2014,3,29
10601848,3045462,1146074,3,1396078210,3,2014,3,29
10601849,2310847,450576,5,1396078950,2,2014,3,29
10601850,2305605,3282501,5,1396079488,2,2014,3,29
10601851,2106488,3643665,2,1396082360,6,2014,3,29


In [30]:
fname = "../datasets/google_local/reviews_timesorted.csv"

reviews_timesorted.to_csv(fname, encoding='utf-8', index=False)